In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F


In [2]:
data = np.load("denoised_data.npy",allow_pickle=True)

In [3]:
X_train, X_test = train_test_split(data.T, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
data_train = scaler.fit_transform(X_train)
data_test = scaler.transform(X_test)

train = torch.tensor(data_train.T, dtype=torch.float32)
test = torch.tensor(data_test.T, dtype=torch.float32)

train_dataset = TensorDataset(train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [4]:
train.shape

torch.Size([400, 4528])

In [5]:
# Define the generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 400),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.model(x)
    
# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(400, 200),
            nn.LeakyReLU(0.2),
            nn.Linear(200, 100),
            nn.LeakyReLU(0.2),
            nn.Linear(100, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)
    
# Create the generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [6]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=400, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=100, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [7]:
# Set up the optimizers and loss function
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
loss_fn = nn.BCELoss()

In [8]:
# Train the GAN
epochs = 5000
for epoch in range(epochs):
    for batch, real_spectra in enumerate(train_loader):
        real_spectra = real_spectra[0].to(device)
        batch_size = real_spectra.size(0)

        # Train the discriminator
        optimizer_D.zero_grad()
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        real_preds = discriminator(real_spectra)
        d_loss_real = loss_fn(real_preds, real_labels)

        noise = torch.randn(batch_size, 100).to(device)
        fake_spectra = generator(noise)
        fake_preds = discriminator(fake_spectra.detach())
        d_loss_fake = loss_fn(fake_preds, fake_labels)

        d_loss = d_loss_real + d_loss_fake

        d_loss.backward()
        optimizer_D.step()

        # Train the generator
        optimizer_G.zero_grad()
        fake_preds_for_generator = discriminator(fake_spectra)
        g_loss = loss_fn(fake_preds_for_generator, real_labels)
        g_loss.backward()
        optimizer_G.step()

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D_loss: {d_loss.item()}, G_loss: {g_loss.item()}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x4528 and 400x200)

In [ ]:

# Save generator and discriminator models
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")

# To generate new spectra using the trained generator
with torch.no_grad():
    noise = torch.randn(1, 100).to(device)
    generated_spectra = generator(noise)

In [1]:
test_dataset = TensorDataset(test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Initialize a list to store the reconstructed spectra
reconstructed_spectra = []

# Generate reconstructed spectra using the generator
generator.eval()
with torch.no_grad():
    for batch, test_spectra in enumerate(test_loader):
        test_spectra = test_spectra[0].to(device)
        batch_size = test_spectra.size(0)
        
        # Generate random noise
        #noise = torch.randn(batch_size, 100).to(device)
        
        # Generate reconstructed spectra
        generated_spectra = generator(noise)
        
        # Save the generated spectra in a list
        reconstructed_spectra.extend(generated_spectra.cpu().numpy())

# Invert the scaling to bring the data back to its original range
reconstructed_spectra = np.array(reconstructed_spectra)
reconstructed_spectra = scaler.inverse_transform(reconstructed_spectra)

NameError: name 'TensorDataset' is not defined